In [1]:

# Importieren der notwendigen Bibliotheken
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import shap
import matplotlib.pyplot as plt
from xgboost import plot_tree
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import tree
import seaborn as sns
acc = []
model = []


# Laden des Datensatzes
file_path = 'Crop_recommendation.csv'  # Stellen Sie sicher, dass sich die Datei im gleichen Verzeichnis befindet
data = pd.read_csv(file_path)

#sns.heatmap(data.corr(),annot=True)

# Features und Zielvariable definieren
X = data[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
y = data['label']

# Zielvariable in numerische Werte kodieren
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=2)

XB = xgb.XGBClassifier()
XB.fit(X_train,y_train)

predicted_values = XB.predict(X_test)

x = metrics.accuracy_score(y_test, predicted_values)
acc.append(x)
model.append('XGBoost')
print("XGBoost's Accuracy is: ", x)

print(classification_report(y_test,predicted_values))

# XGBoost-Modell initialisieren und trainieren
#model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
#model.fit(X_train, y_train)

# Vorhersagen auf den Testdaten
#y_pred = model.predict(X_test)

# Einen Baum visualisieren (z. B. Baum 0)
#plt.figure(figsize=(20, 10))  # Stellen Sie sicher, dass die Grafik groß genug ist
#plot_tree(model, num_trees=0, rankdir='TB')  # num_trees gibt den Index des Baums an
#plt.show()

# SHAP-Explainer erstellen
explainer = shap.Explainer(XB, X_train)

# SHAP-Werte berechnen für die Testdaten
shap_values = explainer(X_test)

# Globaler Überblick: SHAP-Summary-Plot
# Aggregieren der SHAP-Werte über alle Klassen
shap.summary_plot(shap_values, X_test, feature_names=X.columns)

# Beispiel: Lokale Analyse mit SHAP-Force-Plot
sample_idx = 0  # Index eines Beispiels
# Lokale Analyse für eine bestimmte Klasse (z. B. Klasse 0)
shap.force_plot(
    explainer.expected_value[0],  # Erwartungswert der Klasse
    shap_values[sample_idx, :],   # SHAP-Werte des Beispiels
    X_test.iloc[sample_idx, :]    # Eingabemerkmale
)

h:\Meine Ablage\Arbeit\Hochschule\7 Semester\Qualitätssicherung für KI-Systeme\Hausarbeit\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: could not convert string to float: 'rice'